In [ ]:
# Prepare the environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.varmax import VARMAX
import warnings
warnings.filterwarnings('ignore')

# Data Preprocessing

In [ ]:
#Read excel data file
d = pd.read_excel('tele_data_20210819.xlsx')

In [ ]:
from pmdarima.arima import auto_arima
from statsmodels.tsa.stattools import acf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
from pmdarima.arima import ADFTest
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
#Drop the timestamp column and calculate the correlation between each variable
data = d.drop(['ds'],axis=1)
data.corr()

In [ ]:
#Drop target varialbe and leave all the exogenous variable as X
X = data.drop(['y'],axis = 1)
X

In [ ]:
#Define y
y = data.y
y

In [ ]:
pm.plot_acf(y)

In [ ]:
from pmdarima.arima.stationarity import ADFTest

# Test whether we should difference at the alpha=0.05
# significance level
adf_test = ADFTest(alpha=0.05)
p_val, should_diff = adf_test.should_diff(y)  # (0.01, False)
p_val

In [ ]:
#Here we can use rfe to select key exogenous variable that have higher relevance with y
feature_names = X.columns.values
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
rfe = RFE(LinearRegression(), 
          n_features_to_select=5, # number of feature to retain
          step=1 # number of features to eliminate each round
         ).fit(X,y)
feature_names[rfe.get_support()]

In [ ]:
#Drop the variable that was not selected by rfe function
X_new = X.drop(['Producer Price Index by Commodity: Metals and Metal Products: Iron and Steel',
               'Producer Price Index by Commodity: Metals and Metal Products: Copper Wire and Cable'],
                            axis = 1)

In [ ]:
#Use auto_arima function to find combination of q and p with lowest AIC
#FYI: https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.auto_arima.html
model = pm.auto_arima(y=y, X=X_new, 
                      start_p=1, 
                      start_q=1,
                      test='adf',
                      max_p=5, 
                      max_q=5, 
                      m=12,#The m parameter relates to the number of observations per seasonal cycle
                      start_P=0, 
                      seasonal=True,
                      d=0, 
                      D=1, 
                      trace=True,#Whether to print status on the fits.
                      error_action='ignore',
                      suppress_warnings=True
                     )

In [ ]:
print(model.summary())

In [ ]:
model.plot_diagnostics(figsize=(12,8));

In [ ]:
#These are some accuracy metrics we use to compare how accurate our forecast values are.
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax                
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 
            'corr':corr, 'minmax':minmax})

In [ ]:
plt.figure(figsize=(12,5), dpi=100)
plt.xlabel('Date-Monthly')
plt.ylabel('Monthly Earnings')
plt.plot(d['ds'],model.predict_in_sample(X), label='Prediction')
plt.plot(d['ds'],y, label='actual')
plt.title('Comparison between forecast and actual target variable')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
metrics = forecast_accuracy(model.predict_in_sample(X), y)
print('MAPE: %0.5f' % metrics['mape'])
print('MAE: %0.5f' % metrics['mae'])